## ArtificiaL Neural Network

Importing libraries

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Data Preprocessing

In [ ]:
df = pd.read_csv(#data.csv)
df.head()

X = df.iloc[:,3:-1].values
y = df.iloc[:,-1].values

Encoding categorica data

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

One Hot Encoding for features with multiple categories

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[("encoder", OneHotEncoder(), [1])], remainder="passthrough")
X = np.array(ct.fit_transform(X))

Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Building the ANN

Initializing

In [ ]:
ann = tf.keras.models.Sequential()

Adding the input layer and first hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))

Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))

Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

### Training the ANN

Compiling

In [ ]:
ann.compile(optimizer= "adam", loss= "binary_crossentropy", metrics= ["accuracy"])

# "adam" --> Adam algorithm, a stochastic gradient descent method

Training

In [ ]:
ann.fit(X_train, y_train, batch_size= 32, epochs= 100)

### Model Evaluation

In [ ]:
y_pred = ann.predict(X_test)

y_pred = (y_pred > 0.5)

np.concatenate((y_pred.reshape(-1, 1), y_test.reshape(-1, 1)), axis= 1)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(y_test, y_pred)
accuracy_score(y_test, y_pred)

## Convolutional Neural Network

Importing libraries

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

### Data Preprocessing

##### Preprocessing the training set

Image Augmentation - applying series of transformations to images to avoid overfitting.

rescale parameter is for feature scaling, each pixel will be a value between zero and one.

In [ ]:
train_datagen =ImageDataGenerator(
                rescale= 1./255,
                shear_range= 0.2,
                zoom_range= 0.2,
                horizontal_flip= True)

In [ ]:
training_set = train_datagen.flow_from_directory("dataset/training_set", 
                                                 target_size= (64, 64), 
                                                 batch_size= 32, 
                                                 class_mode= "binary")

##### Preprocessing the test set

Apply feature scaling for training and test sets but be careful not to transform/ augment the test set to avoid information leakage

In [ ]:
test_datagen = ImageDataGenerator(rescale= 1./255)

test_set = test_datagen.flow_from_directory("dataset/test_set",
                                           target_size= (64, 64),
                                           batch_size= 32,
                                           class_mode= "binary")

### Building the CNN

##### Initializing the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

##### Adding the convolutional layer

input shape is a list of 3 integers corresponds to the rgb dimension

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters= 32, 
                               kernel_size= 3, 
                               activation= "relu", 
                               input_shape= [64, 64, 3]))

##### Pooling

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size= 2, strides= 2))

##### Adding a second convolutional layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters= 32, 
                               kernel_size= 3, 
                               activation= "relu"))

# remove input_shape parameter

cnn.add(tf.keras.layers.MaxPool2D(pool_size= 2, strides= 2))

##### Flattening

Output is a one dimensional vector that will be the input for the fully connected layer

In [ ]:
cnn.add(tf.keras.layers.Flatten())

##### Full Connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units= 128, activation= "relu"))

##### Output layer

units= 1, activation= "sigmoid" for binary classification

activation= "softmax" for multi-class classification

In [ ]:
cnn.add(tf.keras.layers.Dense(units= 1, activation= "sigmoid"))

### Training the CNN

##### Compiling

In [ ]:
cnn.compile(optimizer= "adam", loss= "binary_crossentropy", metrics= ["accuracy"])

##### Training on the training set and evaluating on the test set

In [ ]:
cnn.fit(x= training_set, validation_data= test_set, epochs= 25)

### Making Prediction

In [ ]:
import numpy as np
from keras.preprocessing import image

test_image = image.load_img("dataset/single_prediction/cat_or_dog_1.jpg", target_size= (64, 64))

# Predict method expects a 2d array
test_image = image.img_to_array(test_image)

# image array must be contained in the same batch size specified when training the data
test_image = np.expand_dims(test_image, axis= 0)

# normalize the image array
test_image = test_image / 255

# identifying class assignment
training_set.class_indices

# cnn predict method
result = cnn.predict(test_image)

if result[0][0] > 0.5:
    prediction = "dog"
else:
    prediction = "cat"
    
print(prediction)